In [83]:
import pickle
import json
import re
from urllib.request import urlopen
from bs4 import BeautifulSoup
import signal
from urllib.parse import urlparse
import uuid

In [2]:
src_train = pickle.load(open('../data/encoded_queries_train.pkl', 'rb'))
src_dev = pickle.load(open('../data/encoded_queries_dev.pkl', 'rb'))

raw_websites = json.load(open('../data/RC_2009-05_webpages.json', 'r'))


corpus_text = []
with open('../data/websites/bow_websites.jsonl', 'r') as f:
    for line in f:
        corpus_text.append(json.loads(line))

src_dev_text = open('../data/bow_queries_dev.tsv', 'r').read().splitlines()


corpus = pickle.load(open('../data/encoded_websites.pkl', 'rb'))


relevance_scores = open('../data/relevance_scores.txt', 'r')
query_webpage_map = {}
for line in relevance_scores:
    split_line = line.split()
    query_webpage_map[int(split_line[0])] = int(split_line[2])


In [11]:
with open("../data_2017-09/RC_2017-09", 'r') as f:
    for i,line in enumerate(f):
        comment = json.loads(line)
        print(comment)
        break

{'author': 'JGMouton4246', 'author_flair_css_class': None, 'author_flair_text': None, 'body': 'Hi, I been a fan of Milovana for a while now, pls send me an invite code in PM. Thanks ', 'can_gild': True, 'collapsed': False, 'collapsed_reason': None, 'controversiality': 0, 'created_utc': 1504224000, 'distinguished': None, 'edited': False, 'gilded': 0, 'id': 'dmehp0v', 'is_submitter': False, 'link_id': 't3_6kkdkq', 'parent_id': 't3_6kkdkq', 'retrieved_on': 1504556642, 'score': 1, 'stickied': False, 'subreddit': 'Milovana', 'subreddit_id': 't5_31hip'}


### Efficient way of creating training data from large comment JSON

In [64]:
'''
Notes
1. There are 83,165,192 comments --> approx 250,000 links in comments
2. Total pass-through ~25 minutes, and we do two pass-throughs
3. en.m.wiki is mapped to en.wiki
'''

valid_domains = {'en.wikipedia.org': True, 'en.m.wikipedia.org': True, 'www.washingtonpost.com': True, 'www.theguardian.com': True, 'www.independent.co.uk': True,
          'www.theatlantic.com': True, 'www.bbc.com': True, 'www.nbcnews.com': True, 'www.usatoday.com': True, 'www.cnn.com': True, 'insider.foxnews.com': True,
          'www.npr.org': True, 'www.espn.com': True, 'www.politico.com': True, 'www.bbc.co.uk': True, 'www.telegraph.co.uk': True, 'www.businessinsider.com': True,
          'www.bloomberg.com': True, 'www.bbc.co.uk': True, 'www.forbes.com': True, 'abcnews.go.com': True, 'www.huffingtonpost.com': True, 'www.latimes.com': True,
          'www.pbs.org': True, 'www.thesun.co.uk': True, 'www.chicagotribune.com': True, 'www.dailymail.co.uk': True, 'www.cnbc.com': True, 'www.foxnews.com': True,
          'www.slate.com': True, 'www.wired.com': True, 'www.investopedia.com': True, 'www.theonion.com': True, 'www.vox.com': True, 'articles.chicagotribune.com': True}

ignore_type = {'pdf': True, 'jpg': True, 'png':True, 'gif':True}

all_urls = {}
valid_urls = {}
id_parent_mapping = {}

DATA_PATH = "../data_2017-09/RC_2017-09"

with open(DATA_PATH, 'r') as f:
    for i,line in enumerate(f):
        comment = json.loads(line)


        id_parent_mapping[comment['id']] = comment['parent_id'][3:]

        cleaned_urls = []
        cleaned_valid_urls = []
        for url in re.findall(r'(https?://\S+-\n)?(?(1)([\S]*)|(https?://\S+))', comment['body']):

            # heuristics for parsing url
            url = "".join(url)
            url = re.sub('\)', '', url)
            url = re.sub('\]', '', url)
            url = re.sub('\n', '', url)
            url = re.sub(',', '', url)
            url = re.sub(' ', '', url)
            if url[-1] == ".":
                url = url[:-1]

            try:
                domain = urlparse(url).netloc
            except:
                continue

            if domain == "en.m.wikipedia.org":
                url = re.sub('\.m', '', url)

            cleaned_urls.append(url)

            if url[-3:] in ignore_type: continue
            
            if domain not in valid_domains: continue
            
            cleaned_valid_urls.append(url)

        if cleaned_urls:
            all_urls[i] = {'urls': cleaned_urls}
        if cleaned_valid_urls:
            valid_urls[i] = {'urls': cleaned_valid_urls, 'id': comment['id']}
            if len(valid_urls) % 10000 == 0: print('Valid URLs collected: ', len(valid_urls))

print('All valid URLs collected')

# build the chains for each valid URL
all_needed_comments = {}
for line in valid_urls:
    id = valid_urls[line]['id']
    chain = []
    while id in id_parent_mapping:
        chain.append(id)
        all_needed_comments[id] = None
        id = id_parent_mapping[id]
    chain = chain[::-1]
    valid_urls[line]['chain'] = chain

print('All id chains constructed')

# do a second pass-through to get all necessary comments
with open(DATA_PATH, 'r') as f:
    for i,line in enumerate(f):
        comment = json.loads(line)
        comment_id = comment['id']
        if comment_id in all_needed_comments:
            all_needed_comments[comment_id] = comment['body']

Valid URLs collected:  10000
Valid URLs collected:  20000
Valid URLs collected:  30000
Valid URLs collected:  40000
Valid URLs collected:  50000
Valid URLs collected:  60000
Valid URLs collected:  70000
Valid URLs collected:  80000
Valid URLs collected:  90000
Valid URLs collected:  100000
Valid URLs collected:  110000
Valid URLs collected:  120000
Valid URLs collected:  130000
Valid URLs collected:  140000
Valid URLs collected:  150000
Valid URLs collected:  160000
Valid URLs collected:  170000
Valid URLs collected:  180000
Valid URLs collected:  190000
Valid URLs collected:  200000
Valid URLs collected:  210000
Valid URLs collected:  220000
Valid URLs collected:  230000
Valid URLs collected:  240000
Valid URLs collected:  250000
All valid URLs collected
All id chains constructed


In [74]:
# save all_needed_comments pkl
# save valid_urls pkl
# save all_urls pkl
pickle.dump(all_needed_comments, open('../data_2017-09/all_needed_comments.pkl', 'wb'))
pickle.dump(valid_urls, open('../data_2017-09/valid_urls.pkl', 'wb'))
pickle.dump(all_urls, open('../data_2017-09/all_urls.pkl', 'wb'))

### Efficient way to scrape and save websites

In [85]:
class TimeoutException(Exception):
    pass

def handler(signum, frame):
    raise TimeoutException

def scrape(url: str) -> str:
    '''
    Simple method to scrape text from URLs. Not very robust. Need to handle exceptions. YouTube links take very long
    '''
    try:
        html = urlopen(url).read()
    except Exception as e:
        return ""

    soup = BeautifulSoup(html, features="html.parser")

    # kill all script and style elements
    for script in soup(["script", "style"]):
        script.extract()    # rip it out

    # get text
    text = soup.get_text()

    # break into lines and remove leading and trailing space on each
    lines = (line.strip() for line in text.splitlines())
    # break multi-headlines into a line each
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    # drop blank lines and short lines
    text = ""
    for chunk in chunks:
        if chunk and len(chunk) > 50:
            # inspired by https://bigscience.huggingface.co/blog/building-a-tb-scale-multilingual-dataset-for-language-modeling#:~:text=Filters%2C%20tools%2C%20and%20indicators%20of%20data%20quality
            only_text = re.sub('[^ \w\*]', '', chunk)
            if len(only_text) / len(chunk) > 0.8:
                text += chunk
    return text


# Conditions
MIN_LENGTH=2

url_file_map = {}
for line in valid_urls:
    if len(valid_urls[line]['chain']) < MIN_LENGTH:
        continue
    for url in valid_urls[line]['urls']:
        url_file_map[url] = {'filename': uuid.uuid4().hex}

pickle.dump(url_file_map, open('../data_2017-09/url_file_map.pkl', 'wb'))

total_urls = len(url_file_map)
print('Total webpages to fetch: ', total_urls)
for i,url in enumerate(url_file_map):
    url_file_map[url]['status'] = 'failure'

    signal.signal(signal.SIGALRM, handler)
    signal.alarm(2) # 5 second timeout
    try:
        text = scrape(url)
    except:
        continue
    if len(text.strip()) > 100:
        with open('../data_2017-09/webpages/' + url_file_map[url]['filename'] + '.txt', 'w') as f:
            f.write(text)
            url_file_map[url]['status'] = 'success'
        
    signal.alarm(0) # disable alarm
    if i > 100: break


Total webpages to fetch:  130193


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


In [ ]:
def get_domain_freqs(urls):
    domains = {}
    for url in urls:
        try:
            domain = urlparse(urls[url]['urls'][0]).netloc
        except:
            print(urls[url]['urls'][0])
        if domain not in domains:
            domains[domain] = 0
        domains[domain] += 1

    sorted_domains = sorted([(domain, domains[domain]) for domain in domains], reverse=True, key=lambda x: x[1])
    for d in sorted_domains[:1000]:
        #if d[0] in valid_domains: continue
        print(d)

get_domain_freqs(valid_urls)

### Misc

In [46]:
src_dev[0]

{'query_id': 5335,
 'encoding': tensor([[ 0.0080,  0.0120, -0.0139,  ..., -0.0024,  0.0485, -0.0249],
         [ 0.0336,  0.0337,  0.0075,  ...,  0.0383,  0.0492,  0.0096]],
        device='cuda:0')}

In [47]:
src_dev_text[0]

'5335\twhat a world it would be without Murdock. [Murdock]( )[Murdoch](http://en.wikipedia.org/wiki/Rupert_Murdoch)'

In [49]:
query_webpage_map[5335]

200

In [5]:
print(corpus_text[200]['contents'][:100])

List of characters from the American TV show The A-TeamThis article lists the characters created for


In [51]:
print(len(corpus))
print(len(corpus_text))

5953
5953


In [13]:
for c in corpus_text:
    web_text = c['contents']
    if "paywall" in web_text: 
        print(web_text[:100])
        print('======')

British weekly news and international affairs publicationFor the profession, see Economist. For othe
John R. Mashey (born 1946) is an American computer scientist, director and entrepreneur.Mashey holds
For a non-technical introduction to the topic, see Introduction to genetics. For other uses, see DNA
System of interlinked hypertext documents accessed over the Internet"WWW" and "The Web" redirect her
This article includes a list of general references, but it lacks sufficient corresponding inline cit
Nonprofit news organization owned by the Church of Christ, ScientistThe Christian Science MonitorThe
Magazine and multi-platform publisher based in Washington, D.C.For the ocean, see Atlantic Ocean. Fo
Type and severity of damage caused by nuclear weaponsThis article includes a list of general referen
This article is about the British newspaper. For the Australian newspaper, see The Daily Telegraph (
Alain de BottonFRSLBorn (1969-12-20) 20 December 1969 (age 52)Zürich, SwitzerlandOccupation